#Data Set Overivew

The tutorial uses the Iowa Liquor Retails Sales. We will be using it dataset to predict future sales for one of the stores

This dataset contains every wholesale purchase of liquor in the State of Iowa by retailers for sale to individuals since January 1, 2012.
The State of Iowa controls the wholesale distribution of liquor intended for retail sale, which means this dataset offers a complete view of retail liquor sales in the entire state. The dataset contains every wholesale order of liquor by all grocery stores, liquor stores, convenience stores, etc., with details about the store and location, the exact liquor brand and size, and the number of bottles ordered.

In [1]:
# https://github.com/hamzafarooq/Time-Series

In [2]:
import numpy as np # linear algebra
import pandas as pd
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import itertools
%matplotlib inline
from matplotlib import pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
import matplotlib
import datetime
import seaborn as sns

sns.set(rc={'figure.figsize':(11, 4)})
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits
from mpl_toolkits.basemap import Basemap

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [ ]:
file = 'Iowa_Liquor_Sales_2023.csv'
df = pd.read_csv(file, header=0)

# Data Overview

In [ ]:
df_backup = df # it is always useful to store an orignial version of the data

In [ ]:
df_rows = df.shape[0]

In [ ]:
# Check for % of missing values in each column - drop nulls
df.isnull().sum()/df_rows*100

In [ ]:
df

In [ ]:
df.dropna(inplace=True)
#df = df.drop('Unnamed: 0',axis=1)

In [ ]:
df.isnull().sum()/df_rows*100

In [ ]:
## check for data types and outliers for continuous variables
df.dtypes

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
#df.drop('Date',axis=1)

In [ ]:
df.describe()

In [ ]:
daily_sale=df.groupby('Date')[['Sale (Dollars)']].sum().sort_values(by = ['Sale (Dollars)'], \
                                                                                ascending=False).reset_index()

daily_sale=daily_sale.sort_values('Date')
daily_sale.head(5)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(daily_sale['Date'],daily_sale['Sale (Dollars)'])
plt.show()

# Finding sales for all Wal-Mart Stores

In [ ]:
df.columns

In [ ]:
df['Store Name'].unique().tolist()

In [ ]:
wmt_list=[]
for i in set(df['Store Name']):
  if 'WAL-MART' in i:
    wmt_list.append(i)

len(wmt_list)

In [ ]:
wmt_stores_only = df[df['Store Name'].isin(wmt_list)]
wmt_stores_only.head(5)

In [ ]:
locations=set(wmt_stores_only['Store Location'])

In [ ]:
def long(q):
  l=[]
  for i in q:
    
    i=i.replace('POINT',"")
    i=i.replace('[',"")
    i=i.replace('(',"")
    k=i.split()
    l.append(float(k[0]))
  return l
    



In [ ]:
def lat(q):
  l=[]
  for i in q:
    i=i.replace('POINT',"")
    i=i.replace('[',"")
    i=i.replace(')',"")
    k=i.split()
    l.append(float(k[1]))
  return l

In [ ]:
wmt_stores_only.dtypes

In [ ]:
wmt_stores_only['lat']= lat(wmt_stores_only['Store Location'].values)
wmt_stores_only['long']= long(wmt_stores_only['Store Location'].values)


In [ ]:
wmt_locations=wmt_stores_only[['Store Number','lat','long']].drop_duplicates()

In [ ]:
wmt_store_sales = wmt_stores_only.groupby('Store Number')[['Sale (Dollars)']].sum(). \
                                          sort_values(by = ['Sale (Dollars)'], ascending=False).reset_index()


In [ ]:
wmt_locations

In [ ]:
wmt_store_sales=pd.merge(wmt_store_sales,wmt_locations , how='left', on='Store Number')

In [ ]:
wmt_store_sales.head(5)

# Exploratory Data Analysis
### Top 5 categories of liquor by total sales

In [ ]:
liquor_sales=wmt_stores_only

In [ ]:
liquor_sales['category_name'] = liquor_sales['category_name'].str.lower().str.title()

In [ ]:
categories = pd.DataFrame(liquor_sales[['category_name','sale_dollars']])
Top_categories = categories.groupby('category_name')[['sale_dollars']].sum().sort_values(by = ['sale_dollars'], \
                                                                                ascending=False).reset_index()

In [ ]:
Top_categories.shape

In [ ]:
Top_categories.head()

In [ ]:
categories_sales = Top_categories.head(10)
categories_sales


### Top Selling 10 categories

In [ ]:
# bar chart for total sales for each category
plt.figure(figsize=(10,10))
sns.barplot(x= 'category_name', y = 'sale_dollars', data=categories_sales)
 
# Rotate x-labels
plt.xticks(rotation=-45)

### Daily Sales

In [ ]:
liquor_sales['year'] = pd.DatetimeIndex(liquor_sales['Date']).year
liquor_sales['month'] = pd.DatetimeIndex(liquor_sales['Date']).month
liquor_sales['day'] = pd.DatetimeIndex(liquor_sales['Date']).day_name()

In [ ]:
daily_sales = liquor_sales.groupby('day')[['sale_dollars']].sum().sort_values(by = ['day']).reset_index()
#daily_sales

In [ ]:
daysofweek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
# bar chart for total sales for each category
plt.figure(figsize=(8,6))
sns.barplot(x= 'day', y = 'sale_dollars', data=daily_sales, order = daysofweek)
 
# Rotate x-labels
plt.xticks(rotation=-45)

### Yearly trend of liquor categories

In [ ]:
yearly_data = pd.DataFrame(liquor_sales[liquor_sales['category_name'].isin(['Canadian Whiskies', 'American Vodkas', \
                                                                            'Spiced Rum','Straight Bourbon Whiskies', \
                                                                            'Tennessee Whiskies'])])

x = yearly_data.groupby(['year', 'category_name'])['sale_dollars'].sum().reset_index()


In [ ]:
plt.figure(figsize=(8,6))
sns.lineplot(x= 'year', y = 'sale_dollars', data=x, hue='category_name')
 
# Rotate x-labels
plt.xticks(rotation=-45)

## Drill deeper into the top selling categories - Canadian Whiskey

In [ ]:
Can_whiskies_items = liquor_sales[liquor_sales['category_name'] == 'Canadian Whiskies']

In [ ]:
Can_whiskies_items['item_description'] = Can_whiskies_items['item_description'].str.lower().str.title()

In [ ]:
Can_whiskies_items.groupby(['item_description'])[['bottles_sold', 'sale_dollars']]\
.sum().sort_values(by = ['bottles_sold', 'sale_dollars'], ascending = False).reset_index().head(5)

In [ ]:
Top_Whisky_items = pd.DataFrame(Can_whiskies_items[Can_whiskies_items['item_description'].isin\
                                      (['Black Velvet', 'Crown Royal', 'Black Velvet Toasted Caramel',\
                                      'Canadian Ltd Whisky','Crown Royal Regal Apple'])])

Top_Whisky_items1 = Top_Whisky_items.groupby(['item_description', 'year'])[['bottles_sold', 'sale_dollars']]\
.sum().sort_values(by = ['item_description', 'year', 'bottles_sold']).reset_index()
#Top_Whisky_items

In [ ]:
len(Top_Whisky_items)

## bottles_sold for Top 5 Canadian Whisky Items

In [ ]:
g = sns.FacetGrid(Top_Whisky_items, col="item_description", col_wrap = 3, height=4)
g.map(sns.pointplot, "year", "bottles_sold", alpha=.1)
g.add_legend();

## Sales for Top 5 Canadian Whisky Items

In [ ]:
b = sns.FacetGrid(Top_Whisky_items, col="item_description", col_wrap = 3, height=4)
b.map(sns.pointplot, "year", "sale_dollars", alpha=.1)
b.add_legend();

## Average Markup for Canadian Whisky Items


In [ ]:
Top_Whisky_items['Average_Markup'] = ((Top_Whisky_items['state_bottle_retail'] - Top_Whisky_items['state_bottle_cost'])/Top_Whisky_items['bottles_sold'])/(Top_Whisky_items['state_bottle_cost']/Top_Whisky_items['bottles_sold'])

In [ ]:
Top_Whisky_items.head()

In [ ]:
Top_Whisky_Markup = Top_Whisky_items.groupby(['item_description'])[['Average_Markup']]\
.sum().sort_values(by = ['Average_Markup'], ascending = False).reset_index()

Top_Whisky_Markup